## Fake News Classifier using Bidirectional LSTM

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('fake-news/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.shape

(20800, 5)

In [7]:
df=df.dropna()

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.15.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [15]:
## vocabulary size
voc_size=5000

In [16]:
messages=X.copy()

In [19]:
messages.head(10)
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
messages.reset_index(inplace=True)

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
   #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2662, 4726, 2654, 2989, 251, 3209, 957, 2052, 2038, 271],
 [1373, 1774, 4706, 1986, 720, 163, 4496],
 [596, 2440, 1383, 3397],
 [1634, 2700, 1300, 1539, 1253, 3845],
 [3326, 720, 3714, 3050, 3035, 3561, 720, 2842, 3047, 4314],
 [824,
  3177,
  2462,
  4825,
  784,
  4484,
  4071,
  4436,
  4533,
  4370,
  3761,
  4221,
  959,
  4839,
  4496],
 [3700, 1719, 2041, 4655, 4299, 3096, 564, 991, 2678, 4993, 4398],
 [2179, 1880, 4381, 2500, 2944, 3817, 4484, 2010, 2678, 4993, 4398],
 [4951, 4085, 2539, 2202, 1406, 2517, 1016, 2206, 4484, 2353],
 [3499, 1985, 3666, 4323, 3803, 2077, 4394, 2964],
 [3028, 2231, 4851, 4361, 774, 1574, 1922, 306, 4592, 449, 4255],
 [1539, 4353, 251, 2517, 4484, 2944],
 [1089, 1777, 486, 16, 1542, 3607, 2442, 1288, 4689],
 [4588, 1596, 4454, 3114, 469, 4970, 4516, 2678, 4993, 4398],
 [1835, 4491, 739, 474, 4214, 2678, 4993, 4398],
 [4790, 2574, 1994, 1622, 663, 4456, 4844, 2479, 4390, 4317],
 [119, 1232, 1774],
 [4724, 1002, 1703, 1842, 4484, 254, 3272, 4496],
 [

## Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 2052, 2038,  271],
       [   0,    0,    0, ...,  720,  163, 4496],
       [   0,    0,    0, ..., 2440, 1383, 3397],
       ...,
       [   0,    0,    0, ..., 2678, 4993, 4398],
       [   0,    0,    0, ..., 3712, 2376, 2366],
       [   0,    0,    0, ..., 3124,  134, 3647]], dtype=int32)

In [28]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 22s 42ms/step - loss: 0.2673 - accuracy: 0.8812 - val_loss: 0.1990 - val_accuracy: 0.9186
Epoch 2/10
383/383 [==============================] - 6s 16ms/step - loss: 0.1329 - accuracy: 0.9486 - val_loss: 0.2283 - val_accuracy: 0.9193
Epoch 3/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0915 - accuracy: 0.9669 - val_loss: 0.2791 - val_accuracy: 0.9150
Epoch 4/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0611 - accuracy: 0.9797 - val_loss: 0.2834 - val_accuracy: 0.9120
Epoch 5/10
383/383 [==============================] - 4s 12ms/step - loss: 0.0359 - accuracy: 0.9881 - val_loss: 0.3797 - val_accuracy: 0.9049
Epoch 6/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0254 - accuracy: 0.9916 - val_loss: 0.4306 - val_accuracy: 0.9079
Epoch 7/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0185 - accuracy: 0.9946 - val_loss: 0.4312 - val_accuracy: 0.9069

In [32]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [33]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 237, 2379]])

In [35]:
print(accuracy_score(y_test,y_pred))

0.9083678541839271
